In [23]:
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt
import string

import nltk
from nltk.corpus import stopwords
from nltk.stem.snowball import SnowballStemmer

from sklearn import preprocessing
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.decomposition import LatentDirichletAllocation

from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer
from time import time
from datetime import date as dt

import warnings
warnings.filterwarnings('ignore')

In [81]:
Time = time()

In [17]:
df = pd.read_csv(r'D:\NAYANA\Term 3\Unstructured\Assignment\Ass1\enron_05_17_2015_with_labels_v2.csv')
print('Training time : {0} sec'.format(round(time()-Time, 3)))
df.head()

Training time : 19.732 sec


,Unnamed: 0,Message-ID,Date,From,To,Subject,X-From,X-To,X-cc,X-bcc,...,Cat_10_level_1,Cat_10_level_2,Cat_10_weight,Cat_11_level_1,Cat_11_level_2,Cat_11_weight,Cat_12_level_1,Cat_12_level_2,Cat_12_weight,labeled
0,0,<18782981.1075855378110.JavaMail.evans@thyme>,2001-05-14 23:39:00,frozenset({'phillip.allen@enron.com'}),frozenset({'tim.belden@enron.com'}),NaN,Phillip K Allen,Tim Belden <Tim Belden/Enron@EnronXGate>,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,False
1,1,<15464986.1075855378456.JavaMail.evans@thyme>,2001-05-04 20:51:00,frozenset({'phillip.allen@enron.com'}),frozenset({'john.lavorato@enron.com'}),Re:,Phillip K Allen,John J Lavorato <John J Lavorato/ENRON@enronXg...,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,False
2,2,<24216240.1075855687451.JavaMail.evans@thyme>,2000-10-18 10:00:00,frozenset({'phillip.allen@enron.com'}),frozenset({'leah.arsdall@enron.com'}),Re: test,Phillip K Allen,Leah Van Arsdall,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,False
3,3,<13505866.1075863688222.JavaMail.evans@thyme>,2000-10-23 13:13:00,frozenset({'phillip.allen@enron.com'}),frozenset({'randall.gay@enron.com'}),NaN,Phillip K Allen,Randall L Gay,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,False
4,4,<30922949.1075863688243.JavaMail.evans@thyme>,2000-08-31 12:07:00,frozenset({'phillip.allen@enron.com'}),frozenset({'greg.piper@enron.com'}),Re: Hello,Phillip K Allen,Greg Piper,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,False


In [19]:
print((df.isnull().sum()/df.shape[0])*100)

print(df.shape)


Unnamed: 0         0.000000
Message-ID         0.000000
Date               0.000000
From               0.000000
To                 4.222450
Subject            3.708342
X-From             0.005605
X-To               1.769034
X-cc              75.089727
X-bcc             99.966177
X-Folder           0.005605
X-Origin           0.005605
X-FileName         0.806338
content            0.223424
user               0.000000
Cat_1_level_1     99.671048
Cat_1_level_2     99.671048
Cat_1_weight      99.671048
Cat_2_level_1     99.708930
Cat_2_level_2     99.708930
Cat_2_weight      99.708930
Cat_3_level_1     99.761307
Cat_3_level_2     99.761307
Cat_3_weight      99.761307
Cat_4_level_1     99.859683
Cat_4_level_2     99.859683
Cat_4_weight      99.859683
Cat_5_level_1     99.931581
Cat_5_level_2     99.931581
Cat_5_weight      99.931581
Cat_6_level_1     99.972942
Cat_6_level_2     99.972942
Cat_6_weight      99.972942
Cat_7_level_1     99.988404
Cat_7_level_2     99.988404
Cat_7_weight      99

In [22]:
final_df = df[['Unnamed: 0', 'Message-ID', 'Date', 'From', 'To', 'Subject', 'X-From', 'X-To', 'X-Folder', 'X-Origin',
               'X-FileName', 'content', 'user', 'labeled']]
final_df.head()

,Unnamed: 0,Message-ID,Date,From,To,Subject,X-From,X-To,X-Folder,X-Origin,X-FileName,content,user,labeled
0,0,<18782981.1075855378110.JavaMail.evans@thyme>,2001-05-14 23:39:00,frozenset({'phillip.allen@enron.com'}),frozenset({'tim.belden@enron.com'}),NaN,Phillip K Allen,Tim Belden <Tim Belden/Enron@EnronXGate>,"\Phillip_Allen_Jan2002_1\Allen, Phillip K.\'Se...",Allen-P,pallen (Non-Privileged).pst,Here is our forecast,allen-p,False
1,1,<15464986.1075855378456.JavaMail.evans@thyme>,2001-05-04 20:51:00,frozenset({'phillip.allen@enron.com'}),frozenset({'john.lavorato@enron.com'}),Re:,Phillip K Allen,John J Lavorato <John J Lavorato/ENRON@enronXg...,"\Phillip_Allen_Jan2002_1\Allen, Phillip K.\'Se...",Allen-P,pallen (Non-Privileged).pst,Traveling to have a business meeting takes the...,allen-p,False
2,2,<24216240.1075855687451.JavaMail.evans@thyme>,2000-10-18 10:00:00,frozenset({'phillip.allen@enron.com'}),frozenset({'leah.arsdall@enron.com'}),Re: test,Phillip K Allen,Leah Van Arsdall,\Phillip_Allen_Dec2000\Notes Folders\'sent mail,Allen-P,pallen.nsf,test successful. way to go!!!,allen-p,False
3,3,<13505866.1075863688222.JavaMail.evans@thyme>,2000-10-23 13:13:00,frozenset({'phillip.allen@enron.com'}),frozenset({'randall.gay@enron.com'}),NaN,Phillip K Allen,Randall L Gay,\Phillip_Allen_Dec2000\Notes Folders\'sent mail,Allen-P,pallen.nsf,"Randy, Can you send me a schedule of the salar...",allen-p,False
4,4,<30922949.1075863688243.JavaMail.evans@thyme>,2000-08-31 12:07:00,frozenset({'phillip.allen@enron.com'}),frozenset({'greg.piper@enron.com'}),Re: Hello,Phillip K Allen,Greg Piper,\Phillip_Allen_Dec2000\Notes Folders\'sent mail,Allen-P,pallen.nsf,Let's shoot for Tuesday at 11:45.,allen-p,False


In [24]:
final_df.dtypes

Unnamed: 0     int64
Message-ID    object
Date          object
From          object
To            object
Subject       object
X-From        object
X-To          object
X-Folder      object
X-Origin      object
X-FileName    object
content       object
user          object
labeled         bool
dtype: object

In [25]:
final_df['Datetime'] = pd.to_datetime(final_df['Date'])
final_df.dtypes

Unnamed: 0             int64
Message-ID            object
Date                  object
From                  object
To                    object
Subject               object
X-From                object
X-To                  object
X-Folder              object
X-Origin              object
X-FileName            object
content               object
user                  object
labeled                 bool
Datetime      datetime64[ns]
dtype: object

# Plot number of emails per Weekday

In [26]:
final_df['Weekday'] = final_df['Datetime'].dt.day_name()
final_df.head()

,Unnamed: 0,Message-ID,Date,From,To,Subject,X-From,X-To,X-Folder,X-Origin,X-FileName,content,user,labeled,Datetime,Weekday
0,0,<18782981.1075855378110.JavaMail.evans@thyme>,2001-05-14 23:39:00,frozenset({'phillip.allen@enron.com'}),frozenset({'tim.belden@enron.com'}),NaN,Phillip K Allen,Tim Belden <Tim Belden/Enron@EnronXGate>,"\Phillip_Allen_Jan2002_1\Allen, Phillip K.\'Se...",Allen-P,pallen (Non-Privileged).pst,Here is our forecast,allen-p,False,2001-05-14 23:39:00,Monday
1,1,<15464986.1075855378456.JavaMail.evans@thyme>,2001-05-04 20:51:00,frozenset({'phillip.allen@enron.com'}),frozenset({'john.lavorato@enron.com'}),Re:,Phillip K Allen,John J Lavorato <John J Lavorato/ENRON@enronXg...,"\Phillip_Allen_Jan2002_1\Allen, Phillip K.\'Se...",Allen-P,pallen (Non-Privileged).pst,Traveling to have a business meeting takes the...,allen-p,False,2001-05-04 20:51:00,Friday
2,2,<24216240.1075855687451.JavaMail.evans@thyme>,2000-10-18 10:00:00,frozenset({'phillip.allen@enron.com'}),frozenset({'leah.arsdall@enron.com'}),Re: test,Phillip K Allen,Leah Van Arsdall,\Phillip_Allen_Dec2000\Notes Folders\'sent mail,Allen-P,pallen.nsf,test successful. way to go!!!,allen-p,False,2000-10-18 10:00:00,Wednesday
3,3,<13505866.1075863688222.JavaMail.evans@thyme>,2000-10-23 13:13:00,frozenset({'phillip.allen@enron.com'}),frozenset({'randall.gay@enron.com'}),NaN,Phillip K Allen,Randall L Gay,\Phillip_Allen_Dec2000\Notes Folders\'sent mail,Allen-P,pallen.nsf,"Randy, Can you send me a schedule of the salar...",allen-p,False,2000-10-23 13:13:00,Monday
4,4,<30922949.1075863688243.JavaMail.evans@thyme>,2000-08-31 12:07:00,frozenset({'phillip.allen@enron.com'}),frozenset({'greg.piper@enron.com'}),Re: Hello,Phillip K Allen,Greg Piper,\Phillip_Allen_Dec2000\Notes Folders\'sent mail,Allen-P,pallen.nsf,Let's shoot for Tuesday at 11:45.,allen-p,False,2000-08-31 12:07:00,Thursday


In [30]:
weekday_df = final_df[(final_df['Weekday'] != 'Sunday') & (final_df['Weekday'] != 'Saturday')]
weekday_df.head(2)

,Unnamed: 0,Message-ID,Date,From,To,Subject,X-From,X-To,X-Folder,X-Origin,X-FileName,content,user,labeled,Datetime,Weekday
0,0,<18782981.1075855378110.JavaMail.evans@thyme>,2001-05-14 23:39:00,frozenset({'phillip.allen@enron.com'}),frozenset({'tim.belden@enron.com'}),NaN,Phillip K Allen,Tim Belden <Tim Belden/Enron@EnronXGate>,"\Phillip_Allen_Jan2002_1\Allen, Phillip K.\'Se...",Allen-P,pallen (Non-Privileged).pst,Here is our forecast,allen-p,False,2001-05-14 23:39:00,Monday
1,1,<15464986.1075855378456.JavaMail.evans@thyme>,2001-05-04 20:51:00,frozenset({'phillip.allen@enron.com'}),frozenset({'john.lavorato@enron.com'}),Re:,Phillip K Allen,John J Lavorato <John J Lavorato/ENRON@enronXg...,"\Phillip_Allen_Jan2002_1\Allen, Phillip K.\'Se...",Allen-P,pallen (Non-Privileged).pst,Traveling to have a business meeting takes the...,allen-p,False,2001-05-04 20:51:00,Friday


In [31]:
weekday_group_by = weekday_df.groupby('Weekday').size().reset_index()
weekday_group_by = weekday_group_by.rename(columns = {0 : 'Count_Emails'})
weekday_group_by

,Weekday,Count_Emails
0,Friday,90185
1,Monday,97276
2,Thursday,96621
3,Tuesday,106515
4,Wednesday,107223


# Find top 10: 
* Mail sender
* Mail  recipients


In [66]:
final_df['From_id'] = final_df['From'].apply(lambda x: str(x).split('{')[1].split('}')[0])
final_df.head()

,Unnamed: 0,Message-ID,Date,From,To,Subject,X-From,X-To,X-Folder,X-Origin,X-FileName,content,user,labeled,Datetime,Weekday,From_id,Hour
0,0,<18782981.1075855378110.JavaMail.evans@thyme>,2001-05-14 23:39:00,frozenset({'phillip.allen@enron.com'}),frozenset({'tim.belden@enron.com'}),NaN,Phillip K Allen,Tim Belden <Tim Belden/Enron@EnronXGate>,"\Phillip_Allen_Jan2002_1\Allen, Phillip K.\'Se...",Allen-P,pallen (Non-Privileged).pst,Here is our forecast,allen-p,False,2001-05-14 23:39:00,Monday,'phillip.allen@enron.com',23
1,1,<15464986.1075855378456.JavaMail.evans@thyme>,2001-05-04 20:51:00,frozenset({'phillip.allen@enron.com'}),frozenset({'john.lavorato@enron.com'}),Re:,Phillip K Allen,John J Lavorato <John J Lavorato/ENRON@enronXg...,"\Phillip_Allen_Jan2002_1\Allen, Phillip K.\'Se...",Allen-P,pallen (Non-Privileged).pst,Traveling to have a business meeting takes the...,allen-p,False,2001-05-04 20:51:00,Friday,'phillip.allen@enron.com',20
2,2,<24216240.1075855687451.JavaMail.evans@thyme>,2000-10-18 10:00:00,frozenset({'phillip.allen@enron.com'}),frozenset({'leah.arsdall@enron.com'}),Re: test,Phillip K Allen,Leah Van Arsdall,\Phillip_Allen_Dec2000\Notes Folders\'sent mail,Allen-P,pallen.nsf,test successful. way to go!!!,allen-p,False,2000-10-18 10:00:00,Wednesday,'phillip.allen@enron.com',10
3,3,<13505866.1075863688222.JavaMail.evans@thyme>,2000-10-23 13:13:00,frozenset({'phillip.allen@enron.com'}),frozenset({'randall.gay@enron.com'}),NaN,Phillip K Allen,Randall L Gay,\Phillip_Allen_Dec2000\Notes Folders\'sent mail,Allen-P,pallen.nsf,"Randy, Can you send me a schedule of the salar...",allen-p,False,2000-10-23 13:13:00,Monday,'phillip.allen@enron.com',13
4,4,<30922949.1075863688243.JavaMail.evans@thyme>,2000-08-31 12:07:00,frozenset({'phillip.allen@enron.com'}),frozenset({'greg.piper@enron.com'}),Re: Hello,Phillip K Allen,Greg Piper,\Phillip_Allen_Dec2000\Notes Folders\'sent mail,Allen-P,pallen.nsf,Let's shoot for Tuesday at 11:45.,allen-p,False,2000-08-31 12:07:00,Thursday,'phillip.allen@enron.com',12


In [68]:
senders = final_df.groupby('From_id').size().reset_index()
senders = sender.rename(columns = {0 : 'Count_Email'})
senders = sender.sort_values(by = 'Count_Email', ascending = False)
print('Top 10 senders : ')
senders.head(10)

Top 10 senders : 


,From_id,Count_Email
10344,'kay.mann@enron.com',16735
19618,'vince.kaminski@enron.com',14368
8757,'jeff.dasovich@enron.com',11411
14960,'pete.davis@enron.com',9149
3342,'chris.germany@enron.com',8801
16941,'sara.shackleton@enron.com',8777
5998,'enron.announcements@enron.com',8587
18525,'tana.jones@enron.com',8490
18127,'steven.kean@enron.com',6759
10260,'kate.symes@enron.com',5438


In [69]:
recipients = final_df.groupby('To').size().reset_index()
recipients = recipients.rename(columns = {0 : 'Count_Email'})
recipients = recipients.sort_values(by = 'Count_Email', ascending= False)
print('Top 10 recipients : ')
recipients.head(10)

Top 10 recipients : 


,To,Count_Email
39455,frozenset({'pete.davis@enron.com'}),9155
50869,frozenset({'tana.jones@enron.com'}),5679
45138,frozenset({'sara.shackleton@enron.com'}),4974
54031,frozenset({'vkaminski@aol.com'}),4870
21782,frozenset({'jeff.dasovich@enron.com'}),4356
26219,frozenset({'kate.symes@enron.com'}),3517
1784,frozenset({'all.worldwide@enron.com'}),3324
33950,frozenset({'mark.taylor@enron.com'}),3298
26550,frozenset({'kay.mann@enron.com'}),3085
16485,frozenset({'gerald.nemec@enron.com'}),3076


# Plot number of emails per Hour per Day

In [49]:
final_df['Hour'] = final_df['Datetime'].dt.hour
final_df.head(2)

,Unnamed: 0,Message-ID,Date,From,To,Subject,X-From,X-To,X-Folder,X-Origin,X-FileName,content,user,labeled,Datetime,Weekday,From_id,Hour
0,0,<18782981.1075855378110.JavaMail.evans@thyme>,2001-05-14 23:39:00,frozenset({'phillip.allen@enron.com'}),frozenset({'tim.belden@enron.com'}),NaN,Phillip K Allen,Tim Belden <Tim Belden/Enron@EnronXGate>,"\Phillip_Allen_Jan2002_1\Allen, Phillip K.\'Se...",Allen-P,pallen (Non-Privileged).pst,Here is our forecast,allen-p,False,2001-05-14 23:39:00,Monday,'phillip.allen@enron.com',23
1,1,<15464986.1075855378456.JavaMail.evans@thyme>,2001-05-04 20:51:00,frozenset({'phillip.allen@enron.com'}),frozenset({'john.lavorato@enron.com'}),Re:,Phillip K Allen,John J Lavorato <John J Lavorato/ENRON@enronXg...,"\Phillip_Allen_Jan2002_1\Allen, Phillip K.\'Se...",Allen-P,pallen (Non-Privileged).pst,Traveling to have a business meeting takes the...,allen-p,False,2001-05-04 20:51:00,Friday,'phillip.allen@enron.com',20


In [70]:
email_hour_day = final_df.groupby(['Weekday','Hour']).size().reset_index()
email_hour_day = email_hour_day.rename(columns = {0 : 'Count_Email'})
email_hour_day = email_hour_day.sort_values(by = 'Count_Email', ascending = False)
email_hour_day.head(10)

,Weekday,Hour,Count_Email
160,Wednesday,16,9407
136,Tuesday,16,9295
135,Tuesday,15,9178
159,Wednesday,15,9034
134,Tuesday,14,8733
158,Wednesday,14,8617
111,Thursday,15,8591
40,Monday,16,8568
39,Monday,15,8505
112,Thursday,16,8397


# Divide the emails in 10 number of topics. Label each topic with top 20 words from each topic

In [51]:
final_df['content'][1]

"Traveling to have a business meeting takes the fun out of the trip. Especially if you have to prepare a presentation. I would suggest holding the business plan meetings here then take a trip without any formal business meetings. I would even try and get some honest opinions on whether a trip is even desired or necessary. As far as the business meetings, I think it would be more productive to try and stimulate discussions across the different groups about what is working and what is not. Too often the presenter speaks and the others are quiet just waiting for their turn. The meetings might be better if held in a round table discussion format. My suggestion for where to go is Austin. Play golf and rent a ski boat and jet ski's. Flying somewhere takes too much time."

In [56]:
print(final_df['content'].isnull().sum())

print(final_df.shape)

# Lets take the column content column

text_with_null = final_df[['content']]
text = text_with_null.dropna()
print(text.shape)
print(text['content'][0])

1156
(517401, 18)
(516245, 1)
Here is our forecast


In [58]:
# Data cleaning

Time = time()
text['Clean_text'] = text['content'].str.lower().str.replace('[^a-z ]', '')
print('Training time : {0} sec\n'.format(round(time()-Time, 3)))

print(text['Clean_text'][0])

Training time : 34.388 sec

here is our forecast


In [59]:
# Removing the stop words

stop = set(stopwords.words('english'))

def sw(text):
    text = [word for word in text.split() if word not in stop]
    return ' '.join(text)

Time = time()
text['Split_words'] = text['Clean_text'].apply(sw)
print('Training time : {0} sec\n'.format(round(time()-Time, 3)))

text.head()

Training time : 32.178 sec



,content,Clean_text,Split_words
0,Here is our forecast,here is our forecast,forecast
1,Traveling to have a business meeting takes the...,traveling to have a business meeting takes the...,traveling business meeting takes fun trip espe...
2,test successful. way to go!!!,test successful way to go,test successful way go
3,"Randy, Can you send me a schedule of the salar...",randy can you send me a schedule of the salary...,randy send schedule salary level everyone sche...
4,Let's shoot for Tuesday at 11:45.,lets shoot for tuesday at,lets shoot tuesday


In [71]:
# DTM

vec = TfidfVectorizer()
Time = time()

DTM = vec.fit_transform(text['Split_words'])
print('Training time : {0} sec\n'.format(round(time()-Time, 3)))


Training time : 100.36 sec



In [72]:
# Model

lda = LatentDirichletAllocation(n_components = 10, max_iter = 15)
Time = time()

lda_output = lda.fit_transform(DTM)
print('Training time : {0} sec\n'.format(round(time()-Time, 3)))

print(lda)

Training time : 2938.308 sec

LatentDirichletAllocation(batch_size=128, doc_topic_prior=None,
                          evaluate_every=-1, learning_decay=0.7,
                          learning_method='batch', learning_offset=10.0,
                          max_doc_update_iter=100, max_iter=15,
                          mean_change_tol=0.001, n_components=10, n_jobs=None,
                          perp_tol=0.1, random_state=None,
                          topic_word_prior=None, total_samples=1000000.0,
                          verbose=0)


In [73]:
# Document-Topic Matrix

topicname = ['Topic' + str(i) for i in range(lda.n_components)]
docname = ['Doc' + str(i) for i in range(DTM.shape[0])]

In [74]:
# I will create the Document topic matrix dataframe

Time = time()
df_doc_topic = pd.DataFrame(np.round(lda_output,2), columns=topicname, index=docname)
print('Training time : {0} sec\n'.format(round(time()-Time, 3)))

df_doc_topic.head(10)

Training time : 0.245 sec



,Topic0,Topic1,Topic2,Topic3,Topic4,Topic5,Topic6,Topic7,Topic8,Topic9
Doc0,0.05,0.05,0.05,0.05,0.55,0.05,0.05,0.05,0.05,0.05
Doc1,0.01,0.01,0.01,0.47,0.01,0.43,0.01,0.01,0.01,0.01
Doc2,0.03,0.03,0.03,0.70,0.03,0.03,0.03,0.03,0.03,0.03
Doc3,0.02,0.02,0.02,0.82,0.02,0.02,0.02,0.02,0.02,0.02
Doc4,0.04,0.04,0.04,0.04,0.04,0.66,0.04,0.04,0.04,0.04
Doc5,0.74,0.03,0.03,0.03,0.03,0.03,0.03,0.03,0.03,0.03
Doc6,0.83,0.02,0.02,0.02,0.02,0.02,0.02,0.02,0.02,0.02
Doc7,0.05,0.05,0.05,0.05,0.05,0.55,0.05,0.05,0.05,0.05
Doc8,0.02,0.21,0.02,0.62,0.02,0.02,0.02,0.02,0.02,0.02
Doc9,0.01,0.75,0.01,0.01,0.01,0.01,0.01,0.01,0.17,0.01


In [75]:
Time = time()
dominating_topic = np.argmax(df_doc_topic.values, axis=1)
df_doc_topic['Dominating_Topic'] = dominating_topic
print('Training time : {0} sec\n'.format(round(time()-Time, 3)))

df_doc_topic.head()

Training time : 0.034 sec



,Topic0,Topic1,Topic2,Topic3,Topic4,Topic5,Topic6,Topic7,Topic8,Topic9,Dominating_Topic
Doc0,0.05,0.05,0.05,0.05,0.55,0.05,0.05,0.05,0.05,0.05,4
Doc1,0.01,0.01,0.01,0.47,0.01,0.43,0.01,0.01,0.01,0.01,3
Doc2,0.03,0.03,0.03,0.70,0.03,0.03,0.03,0.03,0.03,0.03,3
Doc3,0.02,0.02,0.02,0.82,0.02,0.02,0.02,0.02,0.02,0.02,3
Doc4,0.04,0.04,0.04,0.04,0.04,0.66,0.04,0.04,0.04,0.04,5


In [76]:
# Creating the data frame between topic and terms

Time = time()
df_topic_keyword = pd.DataFrame(lda.components_)
print('Training time : {0} sec\n'.format(round(time()-Time, 3)))

# Naming the columns and row
df_topic_keyword.columns = vec.get_feature_names()
df_topic_keyword.index = topicname

df_topic_keyword

Training time : 0.015 sec



,aa,aaa,aaaa,aaaaaa,aaaaaaaa,aaaaaaaaaaa,aaaaaaaaaaaaaaaa,aaaaaaaaaaaaaaaaaa,aaaaaaaaaaaaaaaaaaaaaa,aaaaaaaaaaaaaaaaaaaaaaa,...,zztopwtnet,zzvogewashburnedu,zzy,zzyxpdell,zzyzx,zzz,zzzaolcom,zzzs,zzzz,zzzzs
Topic0,18.759802,0.100146,0.100000,0.360570,0.555485,0.100000,0.299553,0.100000,0.100000,0.100000,...,0.100000,0.100000,0.548261,0.100000,0.100000,0.100000,0.100000,0.100000,0.100000,0.100000
Topic1,34.319740,11.156061,0.100148,0.264391,0.100196,0.100000,0.149026,0.100000,0.100062,0.100000,...,0.100000,0.100000,0.100086,0.486851,0.101986,0.145951,0.100000,0.100174,0.178454,0.109516
Topic2,7.583913,0.100002,0.100000,0.100000,0.100000,0.100000,0.100000,0.224986,0.184471,0.614926,...,0.100000,0.145465,0.100000,0.100000,0.100000,0.100000,0.100000,0.100000,0.100181,0.100000
Topic3,125.201276,2.635906,0.100000,0.100373,0.100000,0.100000,0.100002,0.100122,0.100000,0.100000,...,0.100006,0.100000,0.398566,0.100000,0.100721,0.151330,0.100000,0.100454,0.100530,0.100000
Topic4,0.103498,0.100014,0.100000,0.100000,0.100000,0.100000,0.100000,0.100016,0.100000,0.100000,...,0.502230,0.101589,0.100000,0.100029,0.118745,0.100000,0.100000,0.161699,0.100065,0.100000
Topic5,8.728782,3.354512,0.696563,0.100000,0.100000,0.291035,0.100645,0.100775,0.100234,0.100020,...,0.100000,0.107520,0.100000,0.100045,0.100000,0.100000,0.100023,0.100000,0.266738,0.100000
Topic6,0.100003,0.100005,0.100000,0.100000,0.100000,0.100000,0.100000,0.100000,0.100000,0.100000,...,0.100000,0.100000,0.100000,0.100000,0.100000,0.100000,0.100000,0.100000,0.100000,0.100000
Topic7,2.921760,0.100002,0.100021,0.100000,0.162238,0.100000,0.101518,0.100000,0.100000,0.100000,...,0.100000,0.100000,0.100000,0.100000,0.100000,0.100010,0.296106,0.100000,0.100000,0.100000
Topic8,30.318954,26.777891,0.725279,0.100000,0.100000,0.100000,0.100000,0.100000,0.100000,0.100000,...,0.100000,0.100000,0.100000,0.100000,0.100000,0.100000,0.100000,0.100000,0.100312,0.100000
Topic9,17.455991,0.148005,0.448044,0.100000,0.100000,0.100000,0.100000,0.100000,0.100000,0.100000,...,0.100000,0.100000,0.100000,0.100000,0.100000,0.100000,0.100000,0.100000,0.100000,0.100000


In [77]:
# Finding the top 20 words for each topies

def show_topic(vectorize, model, n_words):
    keywords = np.array(vec.get_feature_names())
    topic_keywords = []
    for topic_weights in lda.components_:
        topic_keywords_locs = (-topic_weights).argsort()[:n_words]
        topic_keywords.append(keywords.take(topic_keywords_locs))
    return topic_keywords

In [78]:
colname = ['R' + str(i) for i in range(20)]

Time = time()
matrix = pd.DataFrame(show_topic(vec, lda, 20), index = topicname, columns = colname)
print('Training time : {0} sec\n'.format(round(time()-Time, 3)))

matrix

Training time : 1.726 sec



,R0,R1,R2,R3,R4,R5,R6,R7,R8,R9,R10,R11,R12,R13,R14,R15,R16,R17,R18,R19
Topic0,deal,thanks,subject,please,kate,deals,pm,cc,know,gas,chris,let,sent,original,message,need,john,daren,meter,questions
Topic1,power,energy,market,would,california,gas,said,enron,new,ferc,price,state,company,us,may,also,prices,electricity,million,customers
Topic2,pm,cc,subject,jeff,forwarded,mark,communicationsenron,communications,richard,please,meeting,call,attached,james,david,john,steven,karen,steve,thanks
Topic3,enron,please,vince,image,email,new,would,business,information,click,meeting,group,time,pm,us,houston,employees,management,thank,program
Topic4,outages,pm,expense,thru,scheduled,sat,report,outage,game,pt,ct,eric,fri,pager,bass,london,server,sun,center,email
Topic5,get,im,going,know,good,message,subject,sent,go,original,dont,email,think,ill,like,thanks,let,hope,time,would
Topic6,final,hourahead,variances,detected,hour,schedulingiso,schedulestxt,parsing,ancillary,awarded,log,schedules,schedule,messages,start,file,date,scid,mkttype,transtype
Topic7,email,message,intended,request,description,named,please,confidential,recipient,sender,information,error,report,privileged,pm,delete,notify,viewing,time,received
Topic8,agreement,please,attached,enron,sara,thanks,fax,legal,corp,subject,america,master,north,phone,cc,mark,credit,eb,houston,smith
Topic9,kay,pm,subject,cc,thanks,vince,please,ben,meeting,sent,forwarded,attached,call,email,original,ge,message,manncorpenronenron,drew,tw


# ---------------------------------------------------- END ---------------------------------------------------